In [1]:

import pandas as pd
import numpy as np
import re
import os
import glob
import bs4
import re
import sys
sys.path.append("../")
from scripts.oldlistings import get_oldlistings_page
import json
from tqdm import tqdm

In [2]:
data_list = []
for i in range(1,6):
    template = f"../data/raw/oldlistings_rent_{i}.json"
    data_list.append(template)
    

In [10]:
data = pd.read_json(data_list[0])

make sure that for meta_data, only 

{'bath', 'type', 'car', 'bed', 'land'}

types are there

In [36]:
def process_df(data_df):
    def find_meta(xs,label, default=None, conversion=lambda x: x):
        for x in xs:
            if x['label'] == label:
                return conversion(x['quantity'])
        return default

    pattern1 = r"\$(\d+)\s*(per week|pw)" 
    
    data = data_df
    
    # extract rented prices to multiple rows
    pdf2 = pd.DataFrame(data['rented_prices'].to_list(),index=data.index).stack()
    pdf2 = pdf2.reset_index()
    pdf2.rename(columns={'level_0':'index',0:'rented_price_dict'},inplace=True)
    pdf2.drop(columns='level_1',inplace=True)
    pdf = pdf2.join(data, on='index', how='left')
    
    # meta data
    pdf['bed'] = pdf['meta_data'].apply(lambda xs: find_meta(xs, 'bed',np.nan))
    pdf['bath'] = pdf['meta_data'].apply(lambda xs: find_meta(xs, 'bath',np.nan))
    pdf['car'] = pdf['meta_data'].apply(lambda xs: find_meta(xs, 'car',np.nan))
    pdf['land'] = pdf['meta_data'].apply(lambda xs: find_meta(xs, 'land',np.nan))
    pdf['type'] = pdf['meta_data'].apply(lambda xs: find_meta(xs, 'type',"No type"))
    
    pdf.drop(columns=['meta_data','rented_prices','index'],inplace=True)

    # extract rented price to multiple columns
    pdf["rented_price"] = pdf["rented_price_dict"].apply(lambda x: re.search(pattern1, x['price']).group(1) if re.search(pattern1, x['price']) else None)
    pdf["date"] = pdf["rented_price_dict"].apply(lambda x: x['date'])
    return pdf

In [1]:
process_df(data).head()

NameError: name 'process_df' is not defined

In [6]:
for k in data_list:
    data = pd.read_json(k)
    labels = []
    for i in range(len(data)):
        for j in data.iloc[i]["meta_data"]:
            if j["label"] not in labels:
                labels.append(j["label"])
                print(labels)



['bed']
['bed', 'bath']
['bed', 'bath', 'car']
['bed', 'bath', 'car', 'land']
['bed', 'bath', 'car', 'land', 'type']
['bed']
['bed', 'bath']
['bed', 'bath', 'car']
['bed', 'bath', 'car', 'land']
['bed', 'bath', 'car', 'land', 'type']
['bed']
['bed', 'bath']
['bed', 'bath', 'car']
['bed', 'bath', 'car', 'type']
['bed', 'bath', 'car', 'type', 'land']
['bed']
['bed', 'bath']
['bed', 'bath', 'car']
['bed', 'bath', 'car', 'type']
['bed', 'bath', 'car', 'type', 'land']
['bed']
['bed', 'bath']
['bed', 'bath', 'car']
['bed', 'bath', 'car', 'type']
['bed', 'bath', 'car', 'type', 'land']


In [27]:
# find the type of each column


# convert the columns to the correct data type
rental_df_2["lat"] = rental_df_2["lat"].astype(float)
rental_df_2["lng"] = rental_df_2["lng"].astype(float)
rental_df_2["rented_price"] = rental_df_2["rented_price"].astype(int)
rental_df_2["date"] = pd.to_datetime(rental_df_2["date"])
rental_df_2["bed"] = rental_df_2["bed"].astype(float)
rental_df_2["bath"] = rental_df_2["bath"].astype(float)
rental_df_2["car"] = rental_df_2["car"].astype(int)
rental_df_2["land"] = rental_df_2["land"].astype(str)
rental_df_2["type"] = rental_df_2["type"].astype(str)

In [3]:
data.iloc[0]["rented_prices"]

[{'date': 'August 2024', 'price': '$800 per week'},
 {'date': 'July 2018', 'price': '$470'},
 {'date': 'July 2018', 'price': '$470 per week'},
 {'date': 'August 2014', 'price': '$450 per week'},
 {'date': 'August 2014', 'price': '$450 pw'},
 {'date': 'July 2013', 'price': '$430 per week'},
 {'date': 'July 2013', 'price': '$430 pw'},
 {'date': 'July 2013', 'price': '$460 per week'},
 {'date': 'July 2013', 'price': '$460 pw'},
 {'date': 'June 2013', 'price': '$460 per week'},
 {'date': 'April 2013', 'price': '$460 per week'},
 {'date': 'April 2013', 'price': '$460 pw'},
 {'date': 'January 2012', 'price': '$450 pw'}]

In [6]:
# extract the date and the price from the rented prices
rented_prices = data["rented_prices"].apply(pd.Series)
rented_prices


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,"{'date': 'August 2024', 'price': '$800 per week'}","{'date': 'July 2018', 'price': '$470'}","{'date': 'July 2018', 'price': '$470 per week'}","{'date': 'August 2014', 'price': '$450 per week'}","{'date': 'August 2014', 'price': '$450 pw'}","{'date': 'July 2013', 'price': '$430 per week'}","{'date': 'July 2013', 'price': '$430 pw'}","{'date': 'July 2013', 'price': '$460 per week'}","{'date': 'July 2013', 'price': '$460 pw'}","{'date': 'June 2013', 'price': '$460 per week'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'date': 'August 2024', 'price': 'POA'}","{'date': 'November 2022', 'price': 'POA'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'date': 'August 2024', 'price': 'POA'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'date': 'August 2024', 'price': '$640 Week'}","{'date': 'June 2024', 'price': '$640'}","{'date': 'May 2023', 'price': '$575'}","{'date': 'February 2022', 'price': '$450'}","{'date': 'September 2021', 'price': '$450'}","{'date': 'August 2014', 'price': '$550 pw'}","{'date': 'February 2013', 'price': '$550 pw'}","{'date': 'July 2012', 'price': '$600 pw'}","{'date': 'May 2011', 'price': '$600'}","{'date': 'May 2011', 'price': '$600 pw'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'date': 'August 2024', 'price': '$800 per week'}","{'date': 'September 2019', 'price': '$560 p.w.'}","{'date': 'September 2019', 'price': '$560 per ...","{'date': 'June 2019', 'price': '$560'}","{'date': 'April 2018', 'price': '$550 per week'}","{'date': 'April 2018', 'price': '$550'}","{'date': 'May 2017', 'price': '$540 per week'}","{'date': 'August 2014', 'price': '$500 per week'}","{'date': 'August 2009', 'price': '$470 pw'}","{'date': 'June 2009', 'price': '$490 pw'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"{'date': 'August 2024', 'price': '$725 per week'}","{'date': 'December 2022', 'price': '$695 per w...","{'date': 'May 2021', 'price': '$650 per week'}","{'date': 'November 2011', 'price': '$650 Weekly'}",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"{'date': 'August 2024', 'price': '$720 per week'}","{'date': 'September 2019', 'price': '$450 per ...","{'date': 'July 2019', 'price': '$500'}","{'date': 'February 2018', 'price': '$470 per w...","{'date': 'February 2018', 'price': '$470'}","{'date': 'November 2016', 'price': '$450 per w...","{'date': 'January 2016', 'price': '$450 per we...","{'date': 'October 2015', 'price': '$450 per we...","{'date': 'November 2015', 'price': '$450 per w...","{'date': 'February 2015', 'price': '$450 per w...",...,"{'date': 'January 2010', 'price': '$415 per we...","{'date': 'October 2007', 'price': 'Contact'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"{'date': 'August 2024', 'price': '$700 - $3,04...","{'date': 'May 2011', 'price': '$560 per week'}","{'date': 'August 2009', 'price': '$560 p.w'}","{'date': 'December 2007', 'price': '$540 p.w'}",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"{'date': 'August 2024', 'price': '$1,350 pw'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"{'date': 'August 2024', 'price': '$725 per week'}","{'date': 'August 2022', 'price': '$575 per week'}","{'date': 'August 2022', 'price': '$575'}","{'date': 'June 2021', 'price': '$450 per week'}","{'date': 'June 2021', 'price': '$450'}","{'date': 'June 2021', 'price': '$475 per week'}","{'date': 'May 2021', 'price': '$475'}","{'date': 'April 2021', 'price': '$475 per week'}","{'date': 'February 2021', 'price': '$550 per w...","{'date': 'February 2021', 'price': '$550'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


: 